In [ ]:
!pip install -qq tensorflow[and-cuda]==2.15.0 tf-keras~=2.15.0 tensorrt-libs==8.6.1 --extra-index-url https://pypi.nvidia.com
!pip install -qq sionna

In [ ]:
from google.colab import drive
drive.mount('/content/drive')#, force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Mar  5 03:58:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.15.0
GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU memory growth enabled")

GPU memory growth enabled


In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, Lambda


In [ ]:

# Configuration (same as your UNet model)
seed = 42
np.random.seed = seed
IMG_WIDTH = 512
IMG_HEIGHT = 1024
IMG_CHANNELS = 3
nfilters = 1024  # Base filter count from your UNet




In [ ]:
# Define dice coefficient metric
def dice_coefficient(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon())

def precision_metric(y_true, y_pred):
    # True positives: element-wise multiplication gives non-zero only where both are non-zero
    true_positives = tf.reduce_sum(y_true * y_pred)

    # All predicted positives
    predicted_positives = tf.reduce_sum(y_pred)

    # Return precision with epsilon to avoid division by zero
    return true_positives / (predicted_positives + tf.keras.backend.epsilon())


def hausdorff_distance(y_true, y_pred, max_dist=1500.0):
    """
    Calculate inverse of the Hausdorff distance between binary masks

    Args:
        y_true: Ground truth binary mask
        y_pred: Predicted binary mask
        max_dist: Maximum distance to use for normalization

    Returns:
        Inverse of the Hausdorff distance (1/HD)
    """
    # Convert predictions to binary
    y_pred_binary = tf.cast(tf.greater_equal(y_pred, 0.5), tf.float32)

    # Find coordinates of non-zero values (foreground pixels)
    y_true_positions = tf.where(tf.greater(y_true, 0))
    y_pred_positions = tf.where(tf.greater(y_pred_binary, 0))

    # If either mask is empty, return zero (maximum distance)
    if tf.equal(tf.shape(y_true_positions)[0], 0) or tf.equal(tf.shape(y_pred_positions)[0], 0):
        return tf.constant(0.0, dtype=tf.float32)

    # Compute all pairwise distances between coordinates
    # From each point in y_true to all points in y_pred
    def minimum_distance(point):
        dists = tf.sqrt(tf.reduce_sum(tf.square(tf.cast(point, tf.float32) -
                                              tf.cast(y_pred_positions, tf.float32)), axis=1))
        return tf.reduce_min(dists)

    # Calculate directed Hausdorff distances
    forward_hausdorff = tf.reduce_max(tf.map_fn(minimum_distance, y_true_positions,
                                              fn_output_signature=tf.float32))

    def minimum_distance_reverse(point):
        dists = tf.sqrt(tf.reduce_sum(tf.square(tf.cast(point, tf.float32) -
                                              tf.cast(y_true_positions, tf.float32)), axis=1))
        return tf.reduce_min(dists)

    # Reverse direction (from y_pred to y_true)
    reverse_hausdorff = tf.reduce_max(tf.map_fn(minimum_distance_reverse, y_pred_positions,
                                              fn_output_signature=tf.float32))

    # Hausdorff distance is the maximum of the two directed distances
    hausdorff_dist = tf.maximum(forward_hausdorff, reverse_hausdorff)

    # Return inverse Hausdorff distance (1/HD), avoiding division by zero
    # Clipping the distance to ensure it's not too small
    hausdorff_dist = tf.clip_by_value(hausdorff_dist, 1e-7, max_dist)
    return hausdorff_dist

In [ ]:

# CNN without skip connections
def build_cnn_no_skip(input_shape):
    """Build a CNN with similar architecture to UNet but without skip connections"""
    inputs = Input(input_shape)
    s = Lambda(lambda x: x / 255)(inputs)

    # Encoder path (same as UNet)
    c1 = Conv2D(int(nfilters/8), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(int(nfilters/8), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(int(nfilters/4), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(int(nfilters/4), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(int(nfilters/2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(int(nfilters/2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(int(nfilters), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(int(nfilters), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(int(nfilters*2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(int(nfilters*2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    # Decoder path (without skip connections)
    u6 = Conv2DTranspose(int(nfilters), (2, 2), strides=(2, 2), padding='same')(c5)
    c6 = Conv2D(int(nfilters), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(int(nfilters), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(int(nfilters/2), (2, 2), strides=(2, 2), padding='same')(c6)
    c7 = Conv2D(int(nfilters/2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(int(nfilters/2), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(int(nfilters/4), (2, 2), strides=(2, 2), padding='same')(c7)
    c8 = Conv2D(int(nfilters/4), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(int(nfilters/4), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(int(nfilters/8), (2, 2), strides=(2, 2), padding='same')(c8)
    c9 = Conv2D(int(nfilters/8), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(int(nfilters/8), (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                 loss='binary_crossentropy',
                 metrics=[dice_coefficient,precision_metric,hausdorff_distance])

    return model



In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    """Data generator for loading images in batches during training."""

    def __init__(self, file_list, input_path, label_path, batch_size=8,
                 img_height=1024, img_width=512, img_channels=3, shuffle=True):
        self.file_list = file_list
        self.input_path = input_path
        self.label_path = label_path
        self.batch_size = batch_size
        self.img_height = img_height
        self.img_width = img_width
        self.img_channels = img_channels
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.file_list))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        """Return the number of batches per epoch"""
        return int(np.ceil(len(self.file_list) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data"""
        # Generate indexes of the batch
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Get list of file names for the batch
        batch_files = [self.file_list[i] for i in batch_indexes]

        # Generate data
        X, y = self._data_generation(batch_files)

        return X, y

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.arange(len(self.file_list))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def _data_generation(self, batch_files):
        """Generate data containing batch_size samples"""
        # Initialize arrays
        X = np.zeros((len(batch_files), self.img_height, self.img_width, self.img_channels), dtype=np.float32)
        y = np.zeros((len(batch_files), self.img_height, self.img_width, 1), dtype=np.float32)

        # Load and process images
        for i, file in enumerate(batch_files):
            # Load input image and normalize to [0,1] range
            img_path = os.path.join(self.input_path, file)
            img = cv2.imread(img_path)[:,:,:self.img_channels]
            X[i] = img.astype(np.float32) / 255.0

            # Load ground truth image
            gt_path = os.path.join(self.label_path, file)
            gt = cv2.imread(gt_path)[:,:,:1]
            y[i] = (gt > 0).astype(np.float32)

        return X, y

# Setup data loading



In [ ]:
!cp '/content/drive/MyDrive/TEM_Rebuttal_Files/JoM_rebuttal/Large_Dataset.zip' Datafiles.zip

In [ ]:


import zipfile

with zipfile.ZipFile('Datafiles.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
# Setup data loading
TRAIN_PATH = '/content/Large_Dataset/1024x512/train/'
TRAIN_PATH_Y = '/content/Large_Dataset/1024x512/ground/'
IMG_HEIGHT = 1024
IMG_WIDTH = 512
IMG_CHANNELS = 3
BATCH_SIZE = 8  # Adjust this based on your GPU memory

# Get file list
train_files = [f for f in os.listdir(TRAIN_PATH) if f.endswith('.png')]
print(f"Found {len(train_files)} training images")

# Create validation split
from sklearn.model_selection import train_test_split
train_files, val_files = train_test_split(train_files, test_size=0.1, random_state=42)

# Create data generators
train_generator = DataGenerator(
    train_files,
    TRAIN_PATH,
    TRAIN_PATH_Y,
    batch_size=BATCH_SIZE,
    img_height=IMG_HEIGHT,
    img_width=IMG_WIDTH,
    img_channels=IMG_CHANNELS
)

val_generator = DataGenerator(
    val_files,
    TRAIN_PATH,
    TRAIN_PATH_Y,
    batch_size=BATCH_SIZE,
    img_height=IMG_HEIGHT,
    img_width=IMG_WIDTH,
    img_channels=IMG_CHANNELS,
    shuffle=False
)


Found 1690 training images


In [ ]:
import pandas as pd

In [ ]:
def train_cnn_model(train_generator, val_generator, epochs=100):
    """Train CNN model without skip connections using data generators"""
    input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

    # Build CNN model
    print("Building CNN without skip connections...")
    cnn_no_skip_model = build_cnn_no_skip(input_shape)
    cnn_no_skip_model.summary()

    # Define callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(
            '1024_512_CNN.hdf5',
            verbose=1,
            save_best_only=True
        )
    ]

    # Train CNN without skip connections using generators
    print("\nTraining CNN without skip connections...")
    cnn_model_history = cnn_no_skip_model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=callbacks
    )

    # Save final model
    cnn_no_skip_model.save('1024_512_CNN.hdf5')
    df_history = pd.DataFrame(cnn_model_history.history)
    df_history.to_csv('model_training_history.csv', index=False)
    print("\nTraining completed. Model weights saved.")
    return cnn_no_skip_model

# Usage:
# Create generators as shown in previous code
train_generator = DataGenerator(train_files, TRAIN_PATH, TRAIN_PATH_Y, batch_size=4)
val_generator = DataGenerator(val_files, TRAIN_PATH, TRAIN_PATH_Y, batch_size=4, shuffle=False)



In [ ]:
# Train the model using generators
cnn_model = train_cnn_model(train_generator, val_generator, epochs=100)

Building CNN without skip connections...
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1024, 512, 3)]    0         
                                                                 
 lambda_2 (Lambda)           (None, 1024, 512, 3)      0         
                                                                 
 conv2d_38 (Conv2D)          (None, 1024, 512, 128)    3584      
                                                                 
 dropout_18 (Dropout)        (None, 1024, 512, 128)    0         
                                                                 
 conv2d_39 (Conv2D)          (None, 1024, 512, 128)    147584    
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 512, 256, 128)     0         
 g2D)                                                            
                  

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


381/381 [==============================] - 310s 565ms/step - loss: 0.0156 - dice_coefficient: 0.0052 - precision_metric: 0.0054 - hausdorff_distance: 5.9147 - val_loss: 0.0092 - val_dice_coefficient: 0.0112 - val_precision_metric: 0.0060 - val_hausdorff_distance: 0.0000e+00
Epoch 2/100
381/381 [==============================] - ETA: 0s - loss: 0.0036 - dice_coefficient: 0.0077 - precision_metric: 0.0083 - hausdorff_distance: 0.0000e+00
Epoch 2: val_loss improved from 0.00921 to 0.00709, saving model to 1024_512_CNN.hdf5
381/381 [==============================] - 173s 453ms/step - loss: 0.0036 - dice_coefficient: 0.0077 - precision_metric: 0.0083 - hausdorff_distance: 0.0000e+00 - val_loss: 0.0071 - val_dice_coefficient: 0.0132 - val_precision_metric: 0.0073 - val_hausdorff_distance: 0.0000e+00
Epoch 3/100
381/381 [==============================] - ETA: 0s - loss: 0.0034 - dice_coefficient: 0.0128 - precision_metric: 0.0139 - hausdorff_distance: 0.0000e+00
Epoch 3: val_loss did not impr